<Center><h1>Computational reproduction, optimisation and discussion of the ATLAS Experiment analysis on the production of ZZ in proton-proton collisions at √s = 13 TeV using Open Data</h1></Center>

### Comparación entre los metodos estadisticos least-square y likelihood implementado en ROOT, para histogramas de estadística baja.

Para evaluar la robustes de ambos metodos, se realizo una serie de procedimientos:
* Se tomo la distribucion de landau(mi modelo), se genero una realización aleatoria con 20000 eventos (“toy sample”) con MPV = 200 y σ = 20.
* Se hizo un Fit.
* Se observo los parametros fiteados, en la cual se tomo para dicho experimento el parametro: MPV que es el valor mas probable.

**Necesitamos incluir algunas bibliotecas estándar de C++ y ROOT**

In [1]:
%jsroot on

In [2]:
#include <TFile.h>
#include <TH1.h>
#include <TLine.h>
#include <TPoint.h>
#include <string>
#include <TMath.h>
#include <stdio.h>
#include <stdlib.h>
#include <cstdio>
#include <iostream>
#include <time.h> 
#include <math.h>

In [3]:
  TCanvas * c1 = new TCanvas("c1" , "c1" , 900 , 600 ) ; 
  int nEvents = 20000

(int) 20000


In [4]:
  TF1 * func = new TF1("func" , "[0]*TMath::Landau(x,[1],[2])",100,500) ; //
  func->SetParameters(nEvents,200,20) ; //pido que el valor mas probable este en 200 y con desviación estandar= 20
  func->SetParNames("N","MPV","Sigma") ;

  TH1D * histo = new TH1D("histo" , "histo" , 20 , 120 , 500 ) ;  
  histo->SetLineColor(2) ; 
  histo->SetMarkerColor(1) ;

  histo->FillRandom("func" , nEvents) ;  

In [5]:
  histo->Fit("func" , "" , "" , 120 , 500 ) ; 
  histo->Draw("e") ;  
  c1->Draw();

 FCN=22.893 FROM MIGRAD    STATUS=CONVERGED      66 CALLS          67 TOTAL
                     EDM=2.19891e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  N            2.01807e+04   2.08700e+02   3.40660e-01  -3.52975e-06
   2  MPV          1.99851e+02   3.10503e-01   5.83757e-04  -1.30201e-03
   3  Sigma        2.04064e+01   1.70373e-01   2.44537e-04  -3.36432e-03


N realizaciones aleatorias

In [6]:
  TH1D * Mean_square_least = new TH1D("Mean_square-least" , "Mean_square-least" , 5000,120,500) ; 
  int valor = 0 ; 

//1000 realizaciones
  for( int index = 0 ; index < 1000 ; ++index){
      //cout<<index<<endl;
      
    //Se generan numeros de eventos que sigan una distribución poissoniana    
    valor = gRandom->Poisson(nEvents);
    func->SetParameters(nEvents,200,20);
      
    //reseteo mi histograma
    histo->Reset(); 
      
    //Usamos FillRandom para llenar aleatoriamente el histograma usando el contenido de la función TF1 que es landau en este caso
    //Se creo y se lleno un histograma (valor) veces con una distribución landau con MPV 200 y sigma 20:
      histo->FillRandom("func" , valor);
      
    //histo->Draw("same"); 
    //Ajustamos el histograma con la misma función predefinida, en este caso es la distribucion de landau, usando: 
    histo->Fit("func" , "Q" , "" , 120 , 500 );
      
    //cout << func->GetParameter(1) << " +/- " << func->GetParError(1) << endl; 
   //mean->Fill( (func->GetParameter(1)-200) /func->GetParError(1) ); 
      
      //Se lleno un histograma con los valores del parametro 1(MPV) de la función ajustada landau, para ver
      //si el valor central es el valor real. 
   Mean_square_least->Fill(func->GetParameter(1)); 
    
 }
  
  Mean_square_least->Draw() ; 

In [7]:
c1->Draw()

In [8]:
Mean_square_least->Fit("gaus","","",198,202);

 FCN=17.2784 FROM MIGRAD    STATUS=CONVERGED      58 CALLS          59 TOTAL
                     EDM=1.54705e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     9.55234e+01   3.74391e+00   6.36408e-03  -1.53129e-04
   2  Mean         1.99972e+02   1.00263e-02   9.53542e-05   5.97816e-03
   3  Sigma        3.12233e-01   7.19389e-03   1.29455e-05  -8.23701e-02


In [9]:
c1->Draw()

In [10]:
  TH1D * Mean_likelihood = new TH1D("Mean_likelihood" , "Mean_likelihood" , 5000,120,500) ; 
  int valor = 0 ; 

//1000 realizaciones
  for( int index = 0 ; index < 1000 ; ++index){
      // cout<<index<<endl;
    valor = gRandom->Poisson(nEvents) ;//
    func->SetParameters(nEvents,200,20) ; 
    histo->Reset(); //reseteo mi histograma
    histo->FillRandom("func" , valor);//lleno mi histograma aleatoriamente con valores  
    //histo->Draw("same"); 
    histo->Fit("func" , "QL" , "" , 120 , 500 );//Ajusto mi función con likelihood(minimiza la función de menos log-likelihood)
    //cout << func->GetParameter(1) << " +/- " << func->GetParError(1) << endl ; 
   // mean->Fill( (func->GetParameter(1)-200) /func->GetParError(1) ) ; 
   Mean_likelihood->Fill( func->GetParameter(1));//lleno mi histograma con el parametro 1 de la función ajustada
    
 }
  
  Mean_likelihood->Draw(); 

In [11]:
c1->Draw()

Se puede ver que el método de log-likelihood tiene la ventaja de tratar correctamente los contenedores con estadísticas bajas, ya que el parametro de MPV esta sesgado cuando se trabaja con el metodo de least-square con poca estadística como se puede ver en el plot de Mean_square-least esto quiere decir que el valor central no es en el valor real de mi modelo, estadisticamente hablando la distribución esta desviada, ademas el RMS de least-square es notablemente mayor que el de log-Likelihood. En caso de estadísticas altas / bin, la distribución del contenido de bin se convierte en una distribución normal y ambos metodos como log-likelihood y least-square dan el mismo resultado. Normalmente, los recuentos de un histograma tienen una distribución de Poisson.

Además, cuando se usa el método square-least, existe el problema de los contenedores con cero entradas, en ROOT, estos contenedores se excluyen del ajuste, pero en realidad contienen información estadística valiosa. En cambio, se consideran en el método de log-likelihood.

El método log-likelihood, es el método recomendado en caso de estadísticas de bin bajas, donde el método least-square puede dar resultados incorrectos, en particular cuando hay varios bins vacíos.

Ver referencias: 

*https://root.cern.ch/doc/master/classTH1.html#a63eb028df86bc86c8e20c989eb23fb2a*

*https://pdg.lbl.gov/2004/reviews/statrpp.pdf*

*https://root-forum.cern.ch/t/the-performance-of-th1-fit-using-chi-square-method/6175*